In [7]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Activation , Dropout
from tensorflow.keras.optimizers import SGD
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

intents_file = open('intents.json').read()
intents = json.loads(intents_file)

In [27]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/aka/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Data PreProcessing

In [28]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [29]:
print(classes)

['greeting', 'goodbye', 'thanks', 'options', 'adverse_drug', 'blood_pressure', 'blood_pressure_search', 'pharmacy_search', 'hospital_search']


In [31]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/aka/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [32]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print(len(documents),'documents')
print(len(classes),'classes',classes)
print(len(words),"unique lemmatized words",words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


#Generating Taining and Testing dataset

In [45]:
training = []
# creating empty array for the output
output_empty =[0]*len(classes)

#training set, bag of words for every sentence 

for doc in documents:
    bag = []
    word_patterns =doc[0]
    
    word_patterns =[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
#creating bag of words with binary values
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1
    training.append([bag,output_row])

random.shuffle(training)
training = np.array(training)

# creating X-Train and Y-Train
train_x = list(training[:,0])
train_y = list(training[:,1])




['hi', 'there']
['how', 'are', 'you']
['is', 'anyone', 'there', '?']
['hey']
['hola']
['hello']
['good', 'day']
['bye']
['see', 'you', 'later']
['goodbye']
['nice', 'chatting', 'to', 'you', ',', 'bye']
['till', 'next', 'time']
['thanks']
['thank', 'you']
['that', "'s", 'helpful']
['awesome', ',', 'thanks']
['thanks', 'for', 'helping', 'me']
['how', 'you', 'could', 'help', 'me', '?']
['what', 'you', 'can', 'do', '?']
['what', 'help', 'you', 'provide', '?']
['how', 'you', 'can', 'be', 'helpful', '?']
['what', 'support', 'is', 'offered']
['how', 'to', 'check', 'adverse', 'drug', 'reaction', '?']
['open', 'adverse', 'drug', 'module']
['give', 'me', 'a', 'list', 'of', 'drug', 'causing', 'adverse', 'behavior']
['list', 'all', 'drug', 'suitable', 'for', 'patient', 'with', 'adverse', 'reaction']
['which', 'drug', 'dont', 'have', 'adverse', 'reaction', '?']
['open', 'blood', 'pressure', 'module']
['task', 'related', 'to', 'blood', 'pressure']
['blood', 'pressure', 'data', 'entry']
['i', 'want',

/home/aka/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [49]:
model = Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation = 'softmax'))

In [51]:
sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=["accuracy"])

In [53]:
history = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,
                   verbose=1)
model.save('charbot_model.h5',history)


Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.2199 - accuracy: 0.0854
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1305 - accuracy: 0.1839
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0987 - accuracy: 0.2339
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0475 - accuracy: 0.3450
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 1.8374 - accuracy: 0.4299
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.7790 - accuracy: 0.3287
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.6174 - accuracy: 0.4898
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.4634 - accuracy: 0.6052
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2542 - accuracy: 0.6197
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2416 - accuracy: 0.6152
Epoch 11/

In [59]:
model.save('chatbot_model.h5',history)

In [60]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))
def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
#Creating tkinter GUI
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
        ints = predict_class(msg)
        res = getResponse(ints, intents)
        ChatBox.insert(END, "Bot: " + res + '\n\n')           
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)
root = Tk()
root.title("Chatbot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatBox.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )
#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
root.mainloop()

TclError: no display name and no $DISPLAY environment variable